In [1]:
import pandas as pd
import requests 

In [10]:
r = requests.get("http://localhost:5001/")
print(r.json())

r = requests.get("http://localhost:5001/hello?name=Shiv")
print(r.json())

{'message': 'api: success'}
{'message': 'Hello Shiv!'}


In [7]:
df_col = pd.read_csv('survery_column_length.csv')
df_survey = pd.read_csv('survey_results_public.csv')

df_survey_info = pd.DataFrame({
    "Column Name": df_survey.columns,
    "Non-Null": df_survey.notnull().sum().values,
    "Dtype": df_survey.dtypes.values
})

df_survey_info = pd.merge(df_col, df_survey_info, on='Column Name', how='left')

rows = len(df_survey)

constraints = []
for i,row in df_survey_info.iterrows():
    col = row['Column Name'].replace(' ', '_')
    s = f'''{col}'''
    
    if row['Non-Null'] == 0:
        s+= ' text'
    elif row['Dtype'] == 'object':
        if row['Max Length'] < 100:
            s+= f''' varchar(255)'''
        else:
            s+= f''' text'''
    elif row['Dtype'] == 'int64':
        s+= ' int'
    elif row['Dtype'] == 'float64':
        s+= ' numeric'
    
    if row['Non-Null'] == rows:
        s+= ' not null'
    constraints.append(s)

print(',\n'.join(constraints))

{'message': 'Hello Shiv!'}

In [2]:
df = pd.read_csv('../spot_cleaned_20250409_004052.csv')

In [12]:
import requests
import base64
from urllib.parse import urlencode
import time
import pandas as pd
import copy
from datetime import datetime

client_id = 'd2d5bf12b79f4947be8568a2a4105a93'
client_secret = '61ec80164ca14449a0588e4e64330fda'

# Encode client_id and client_secret in Base64
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

# Set request headers
headers1 = {
    "Authorization": f"Basic {auth_header}",
    "Content-Type": "application/x-www-form-urlencoded",
}

# Set request payload
data = {
    "grant_type": "client_credentials"
}

# Send POST request
response = requests.post("https://accounts.spotify.com/api/token", headers=headers1, data=data)

# Handle response
if response.status_code == 200:
    token = response.json().get("access_token")
    print("Access Token:", token)
    tokenType = response.json().get("token_type")
    print("Token Type: ", tokenType)
    expiresIn = response.json().get("expires_in")
    print("Expires In: ", expiresIn)
else:
    print("Error:", response.status_code, response.text)
print()

# Set the base URLs and search parameters
search_url = "https://api.spotify.com/v1/search"

headers = {"Authorization": f"Bearer {token}"}


# A couple good search queries are below
# Top 1000 greatest songs of all time - 3Q1DIJ51dJpUO6RhnIHdVx
# Billboard Hot 100: All #1 hit songs 1958-2024 - 7c2c13pKxvFDSV4WSyydyg

playlist_id = "3Q1DIJ51dJpUO6RhnIHdVx"

# Get tracks from the playlist (handling pagination)
tracks_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
track_data = []
all_track_details = []
track_count = 0

while tracks_url:
    time.sleep(1)
    response = requests.get(tracks_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        track_items = data.get("items", [])

        for track_info in track_items:
            track = track_info.get("track", {})
            track_name = track.get("name", "Unknown Track")
            track_id = track.get("id", "No ID")
            artist_name = ", ".join([artist["name"] for artist in track.get("artists", [])])
            duration_ms = track.get("duration_ms", 0)

            track_data.append({
                "Track Name": track_name,
                "Track ID": track_id,
                "Artist(s)": artist_name,
                "Duration (ms)": duration_ms
            })

            track_details = {key: track.get(key, "Unknown") for key in track.keys()}

            track_details.pop("preview_url", None)

            artists = track.get("artists", [])
            artist_names = [artist.get("name", "Unknown Artist") for artist in artists]
            track_details["artists"] = ", ".join(artist_names)

            track_details["available_markets"] = ", ".join(track.get("available_markets", []))

            album = track.get("album", {})
            for album_key, album_value in album.items():
                col_name = f"album_{album_key}"
                if isinstance(album_value, list):
                    if all(isinstance(i, dict) for i in album_value):
                        track_details[col_name] = "; ".join(
                            [str({k: v for k, v in item.items()}) for item in album_value]
                        )
                    else:
                        track_details[col_name] = ", ".join(map(str, album_value))

                elif isinstance(album_value, dict):
                    # Flatten dicts as a string
                    track_details[col_name] = ", ".join(f"{k}: {v}" for k, v in album_value.items())

                else:
                    # Keep simple values
                    track_details[col_name] = album_value
            track_details.pop("album", None)

            track_details["external_id_isrc"] = track.get("external_ids", {}).get("isrc", "")
            track_details.pop("external_ids", None)

            track_details["spotify_url"] = track.get("external_urls", {}).get("spotify", "")
            track_details.pop("external_urls", None)


            all_track_details.append(track_details)

            track_count += 1
            print(f"{track_count}. {track_name} - {artist_name} (Duration: {duration_ms} ms)")

        tracks_url = data.get("next")

    else:
        print("Error fetching tracks:", response.status_code)
        break

print(f"\nTotal Tracks Retrieved: {track_count}")

df = pd.DataFrame(all_track_details)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
spot_file = f"spot_tracks_{timestamp}.csv"
df.to_csv(spot_file, index=False)

spot_df = df.copy()

cols_to_transform = ['artists', 'name', 'album_name']

for col in cols_to_transform:
    if col in spot_df.columns:
        spot_df[col] = spot_df[col].str.lower().str.strip()

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
spot_cleaned = f"spot_cleaned_{timestamp}.csv"
spot_df.to_csv(spot_cleaned, index=False)




Access Token: BQDt4paqZyTrNsT4-WmurHuIPsRCm-nOBGu0NjHrz0H-lYHG0Z3TQnJENzZ7Oo99JJrTY3ZACOEz3FDnAfgBaadSlSXL1-lcbZXAHcOmqXLx9QbFCv7zP62sqCOZ6u_X7XEGrKeqwSI
Token Type:  Bearer
Expires In:  3600

1. Don't Change - INXS (Duration: 268200 ms)
2. Halo - Beyoncé (Duration: 261640 ms)
3. It's My Life - Bon Jovi (Duration: 224493 ms)
4. Where Did Our Love Go - The Supremes (Duration: 153333 ms)
5. Ordinary World - Duran Duran (Duration: 340200 ms)
6. Black or White - Michael Jackson (Duration: 254933 ms)
7. Don't Get Me Wrong - Pretenders (Duration: 226066 ms)
8. Running Up That Hill (A Deal With God) - Kate Bush (Duration: 298933 ms)
9. Hall of Fame (feat. will.i.am) - The Script, will.i.am (Duration: 202533 ms)
10. Eye of the Tiger - Survivor (Duration: 245640 ms)
11. Feel - Robbie Williams (Duration: 263866 ms)
12. I'll Be There for You - Theme From "Friends" - The Rembrandts (Duration: 188359 ms)
13. Music Sounds Better With You - Stardust, Benjamin Diamond, Alan Braxe, Thomas Bangalter (Du

In [13]:
spot_df2 = spot_df.copy()

,available_markets,explicit,type,episode,track,artists,disc_number,track_number,duration_ms,href,...,album_images,album_name,album_release_date,album_release_date_precision,album_uri,album_artists,album_external_urls,album_total_tracks,external_id_isrc,spotify_url
0,,False,track,False,True,ricky nelson,1,27,153933,https://api.spotify.com/v1/tracks/33FPsMEl3Uwp...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",ricky / ricky nelson,2001-06-07,day,spotify:album:2KaqhTuykw9yrstJJMgiBe,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/2KaqhT...,31,USEM30100494,https://open.spotify.com/track/33FPsMEl3UwpytD...
1,"AR, AU, AT, BE, BO, BR, BG, CL, CO, CY, CZ, DK...",False,track,False,True,domenico modugno,1,1,219120,https://api.spotify.com/v1/tracks/5zyrEv4F3FaL...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",nel blu dipinto di blu,2008-04-28,day,spotify:album:490sLjsTmSo7MnA8noKMOH,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/490sLj...,14,USA560766736,https://open.spotify.com/track/5zyrEv4F3FaLECI...
2,"CA, US",False,track,False,True,the elegants,1,1,162773,https://api.spotify.com/v1/tracks/3c7KT5CN8uYR...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",little star: the best of the elegants,2005-07-26,day,spotify:album:3HMkMDhnu5RgwbUgfRX9WO,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/3HMkMD...,12,USRH10300788,https://open.spotify.com/track/3c7KT5CN8uYRaK3...
3,,False,track,False,True,tommy edwards,1,1,156440,https://api.spotify.com/v1/tracks/01OFUZ8btJxF...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",the best of tommy edwards,1997-01-01,day,spotify:album:2u41G8wCYWcAvTAdUvXlj4,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/2u41G8...,10,NLF055890007,https://open.spotify.com/track/01OFUZ8btJxFI6n...
4,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,conway twitty,1,1,133826,https://api.spotify.com/v1/tracks/6neFMH7Beu1u...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",the #1 hits collection,2000-01-01,day,spotify:album:1cE3wPf9E328RukGpUp38O,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/1cE3wP...,33,USPR35910034,https://open.spotify.com/track/6neFMH7Beu1uVcs...


In [18]:
spot_df2[spot_df2['external_id_isrc'].isin(spot_df1['external_id_isrc'].tolist())]

,available_markets,explicit,type,episode,track,artists,disc_number,track_number,duration_ms,href,...,album_images,album_name,album_release_date,album_release_date_precision,album_uri,album_artists,album_external_urls,album_total_tracks,external_id_isrc,spotify_url
3,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,the supremes,1,1,153333,https://api.spotify.com/v1/tracks/69RH84na5iUN...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",where did our love go,1964-08-31,day,spotify:album:05pI1Rx1HQ4KA0a0e3PJlV,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/05pI1R...,12,USMO19290049,https://open.spotify.com/track/69RH84na5iUNwrw...
9,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,survivor,1,4,245640,https://api.spotify.com/v1/tracks/2KH16WveTQWT...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",rocky iv,1985,year,spotify:album:3t3BbpFJiGcXl4jI5CRLLA,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/3t3Bbp...,11,USVR19600010,https://open.spotify.com/track/2KH16WveTQWT6KO...
14,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,kool & the gang,1,1,298866,https://api.spotify.com/v1/tracks/3K7Q9PHUWPTa...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",celebrate!,1980-09-29,day,spotify:album:2kc4mhFRsoIRVD0XEYnwhI,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/2kc4mh...,8,USPR38007190,https://open.spotify.com/track/3K7Q9PHUWPTaknl...
16,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,the escape club,1,1,244826,https://api.spotify.com/v1/tracks/4Qn1QhHhUT01...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",wild wild west,2008-10-01,day,spotify:album:4OTNu8Vo0dYxX1UdUYLx1y,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/4OTNu8...,2,USA560852675,https://open.spotify.com/track/4Qn1QhHhUT01Lfp...
20,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,phil collins,1,10,206360,https://api.spotify.com/v1/tracks/63CHa6rmamv9...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",the singles (expanded),2016-10-14,day,spotify:album:7yZHLfxqiGPbSQLrVJljah,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/7yZHLf...,45,USRH11603367,https://open.spotify.com/track/63CHa6rmamv9Ose...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,madonna,1,3,218626,https://api.spotify.com/v1/tracks/1ZPlNanZsJSP...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",like a virgin,1984-11-12,day,spotify:album:2IU9ftOgyRL2caQGWK1jjX,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/2IU9ft...,11,USWB10002748,https://open.spotify.com/track/1ZPlNanZsJSPK5h...
991,"AR, AU, AT, BE, BO, BR, BG, CL, CO, CR, CY, CZ...",False,track,False,True,bryan adams,1,12,393640,https://api.spotify.com/v1/tracks/1Eb90nmqTrxy...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",waking up the neighbours,1991-09-24,day,spotify:album:6Rk2VghhfqeYrgKxS21eBh,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/6Rk2Vg...,15,USAM19190012,https://open.spotify.com/track/1Eb90nmqTrxylKF...
995,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,whitney houston,1,1,271093,https://api.spotify.com/v1/tracks/4eHbdreAnSOr...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",the bodyguard - original soundtrack album,1992-11-17,day,spotify:album:7JVJlkNNobS0GSoy4tCS96,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/7JVJlk...,12,USAR19200110,https://open.spotify.com/track/4eHbdreAnSOrDDs...
1003,,False,track,False,True,dexys midnight runners,1,3,240693,https://api.spotify.

In [25]:
spot_df1.query("external_id_isrc=='USMO19290049'")

,available_markets,explicit,type,episode,track,artists,disc_number,track_number,duration_ms,href,...,album_images,album_name,album_release_date,album_release_date_precision,album_uri,album_artists,album_external_urls,album_total_tracks,external_id_isrc,spotify_url
111,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,the supremes,1,1,153333,https://api.spotify.com/v1/tracks/69RH84na5iUN...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",where did our love go,1964-08-31,day,spotify:album:05pI1Rx1HQ4KA0a0e3PJlV,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/05pI1R...,12,USMO19290049,https://open.spotify.com/track/69RH84na5iUNwrw...


In [31]:
spot_df2.query("name=='thriller'")

,available_markets,explicit,type,episode,track,artists,disc_number,track_number,duration_ms,href,...,album_images,album_name,album_release_date,album_release_date_precision,album_uri,album_artists,album_external_urls,album_total_tracks,external_id_isrc,spotify_url
465,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,michael jackson,1,2,358053,https://api.spotify.com/v1/tracks/7azo4rpSUh8n...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",scream,2017-09-27,day,spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/2X8UOI...,14,USSM19902989,https://open.spotify.com/track/7azo4rpSUh8nXgt...


In [30]:
spot_df2.query('name=="thriller"')

,available_markets,explicit,type,episode,track,artists,disc_number,track_number,duration_ms,href,...,album_images,album_name,album_release_date,album_release_date_precision,album_uri,album_artists,album_external_urls,album_total_tracks,external_id_isrc,spotify_url
465,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",False,track,False,True,michael jackson,1,2,358053,https://api.spotify.com/v1/tracks/7azo4rpSUh8n...,...,"{'height': 640, 'url': 'https://i.scdn.co/imag...",scream,2017-09-27,day,spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk,{'external_urls': {'spotify': 'https://open.sp...,spotify: https://open.spotify.com/album/2X8UOI...,14,USSM19902989,https://open.spotify.com/track/7azo4rpSUh8nXgt...


In [29]:
spot_df2.columns

Index(['available_markets', 'explicit', 'type', 'episode', 'track', 'artists',
       'disc_number', 'track_number', 'duration_ms', 'href', 'id', 'name',
       'popularity', 'uri', 'is_local', 'album_available_markets',
       'album_type', 'album_album_type', 'album_href', 'album_id',
       'album_images', 'album_name', 'album_release_date',
       'album_release_date_precision', 'album_uri', 'album_artists',
       'album_external_urls', 'album_total_tracks', 'external_id_isrc',
       'spotify_url'],
      dtype='object')

In [32]:
import requests
import time
import pandas as pd

lastfm_key = "785f1b57835250d3bec5b0c2682b32aa"
API_KEY = '785f1b57835250d3bec5b0c2682b32aa'
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

import requests
import time

tracks_per_tag = 150
limit_per_page = 150
max_tags = 25


top_tags_url = f'http://ws.audioscrobbler.com/2.0/?method=tag.getTopTags&api_key={API_KEY}&format=json'
response = requests.get(top_tags_url)
top_tags_data = response.json()
top_tags = [tag['name'] for tag in top_tags_data['toptags']['tag']]

#top_tags = [(tag['name'], tag['count']) for tag in top_tags_data['toptags']['tag']]
#print("Top tags on Last.fm: \n")
#for i, tag in enumerate(top_tags_data['toptags']['tag'], 1):
    #print(f"{i}. {tag['name']}, {tag['count']}")
print()

def tag_get_top_tracks(tag):
    tag_tracks = []
    page = 1

    while len(tag_tracks) < tracks_per_tag:
        params = {
            "method": "tag.getTopTracks",
            "tag": tag,
            "api_key": API_KEY,
            "format": "json",
            "limit": limit_per_page,
            "page": page
        }

        response = requests.get(BASE_URL, params=params).json()
        request_url = f"{BASE_URL}?" + "&".join([f"{key}={value}" for key, value in params.items()])
        print(f"Request URL: {request_url}")

        tracks = response.get("tracks", {}).get("track", [])
        attr = response.get("tracks", {}).get("@attr", {})
        
        if not tracks:
            break

        tag_tracks.extend(tracks)
        print(f"{tag} – page {page}, total pages: {attr.get('totalPages')}, tracks returned: {len(tracks)}")

        page += 1
        time.sleep(0.25)

    return tag_tracks[:tracks_per_tag]


for tag in top_tags[:max_tags]:
    all_tracks = {}

    print(f"Fetching top {tracks_per_tag} tracks for tag: {tag}")
    tracks_data = tag_get_top_tracks(tag)
    print(f"Fetched {len(tracks_data)} tracks for tag: {tag}\n")

    all_tracks[tag] = tracks_data
    
    time.sleep(0.5)

print()

"""
for tag, tracks in all_tracks.items():
    print(f"\nTop 10 tracks for '{tag}':")
    for track in tracks[:10]:
        print(f"{track['name']} - {track['artist']['name']}")
"""




Fetching top 150 tracks for tag: rock
Request URL: http://ws.audioscrobbler.com/2.0/?method=tag.getTopTracks&tag=rock&api_key=785f1b57835250d3bec5b0c2682b32aa&format=json&limit=150&page=1
rock – page 1, total pages: 3464, tracks returned: 50
Request URL: http://ws.audioscrobbler.com/2.0/?method=tag.getTopTracks&tag=rock&api_key=785f1b57835250d3bec5b0c2682b32aa&format=json&limit=150&page=2
rock – page 2, total pages: 3464, tracks returned: 300
Fetched 150 tracks for tag: rock

Fetching top 150 tracks for tag: electronic
Request URL: http://ws.audioscrobbler.com/2.0/?method=tag.getTopTracks&tag=electronic&api_key=785f1b57835250d3bec5b0c2682b32aa&format=json&limit=150&page=1
electronic – page 1, total pages: 2600, tracks returned: 150
Fetched 150 tracks for tag: electronic

Fetching top 150 tracks for tag: seen live
Request URL: http://ws.audioscrobbler.com/2.0/?method=tag.getTopTracks&tag=seen live&api_key=785f1b57835250d3bec5b0c2682b32aa&format=json&limit=150&page=1
seen live – page 1,

'\nfor tag, tracks in all_tracks.items():\n    print(f"\nTop 10 tracks for \'{tag}\':")\n    for track in tracks[:10]:\n        print(f"{track[\'name\']} - {track[\'artist\'][\'name\']}")\n'

In [46]:
lastfm_data = pd.DataFrame(all_tracks['Progressive rock'])

In [101]:
lookup = list(lastfm_data[~lastfm_data['mbid'].replace('', pd.NA).isna()]['mbid'].sample(10).values)
lookup

['ee5b5d16-3657-4ce9-8034-d239e5bcb3f7',
 'eae8685c-0a07-4636-8e7a-153c33affe60',
 'b21327fd-5406-45b5-9649-8d52a7f3171e',
 '9c08d02d-5e49-4ef9-b538-dce167adc54a',
 'cbdbf977-7ac0-42ee-ab9d-e64ed653d1da',
 '715a5869-567d-4d8b-b504-67d784b6b955',
 '8e8b3251-c49c-404c-b2f3-4133078df6bd',
 '89af5e8f-8e03-4cb9-a923-adada0faa995',
 'cbbe5ec2-2c93-4cd4-a158-39cac7fa5850',
 '2e7c919c-cd21-46a8-99a6-aa0c733dabaf']

In [105]:
base_url = 'https://acousticbrainz.org'
endpoint = f"{base_url}/api/v1/low-level"
    
params = {
    'recording_ids': lookup,
    'MAX_ITEMS_PER_BULK_REQUEST':10
}

response = requests.get(endpoint, params=params)

In [106]:
params

{'recording_ids': ['ee5b5d16-3657-4ce9-8034-d239e5bcb3f7',
  'eae8685c-0a07-4636-8e7a-153c33affe60',
  'b21327fd-5406-45b5-9649-8d52a7f3171e',
  '9c08d02d-5e49-4ef9-b538-dce167adc54a',
  'cbdbf977-7ac0-42ee-ab9d-e64ed653d1da',
  '715a5869-567d-4d8b-b504-67d784b6b955',
  '8e8b3251-c49c-404c-b2f3-4133078df6bd',
  '89af5e8f-8e03-4cb9-a923-adada0faa995',
  'cbbe5ec2-2c93-4cd4-a158-39cac7fa5850',
  '2e7c919c-cd21-46a8-99a6-aa0c733dabaf'],
 'MAX_ITEMS_PER_BULK_REQUEST': 10}

In [108]:
len(response.json())

2

In [72]:
response.re

<Response [200]>